In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from preprocess import *

In [9]:
df

,sentences,labels,cleaned
0,Merrill Lynch analyst Campbell Morgan upgraded his recommendation on PaperlinX from `` neutral '' to `` buy '' in May .,2,merrill lynch analyst campbell morgan upgraded recommendation paperlinx from neutral to buy may
1,Eriikka S+Âderstr+Âm has previously held several positions in finance and control at Nokia Networks including acting as the Business Group Controller and having the corporate controller position at Nokia Siemens Networks .,1,eriikka s+âderstr+âm previously held several positions finance control nokia networks including acting business group controller corporate controller position nokia siemens networks
2,The webcast may be followed online on the company website at www.ruukki.com/investors .,1,webcast may followed online company website www.ruukki.com/investors
3,"Typical end-uses include roof structures , floorings , walls and ceilings , non-visible structures in vehicles , packaging and boxes , construction site structures , fencing and shelters , and formwork with a limited number of concrete pourings .",1,typical end-uses include roof structures floorings walls ceilings non-visible structures vehicles packaging boxes construction site structures fencing shelters formwork limited number concrete pourings
4,"The sale will be finalized in September or October , the company said .",1,sale finalized september october company said
...,...,...,...
1803,With this appointment Kaupthing Bank aims to further co-ordinate Capital Markets activities within the Group and to improve the overall service to clients .,2,appointment kaupthing bank aims to further co-ordinate capital markets activities within group to improve overall service to clients
1804,"Jon Risfelt has previously held operational executive positions in Europolitan , Ericsson , SAS , American Express card and travel divisions , as well as Nyman & Schultz ( CEO ) , Vodafone Sweden ( CEO ) , and Gambro Renal Products ( CEO ) .",1,jon risfelt previously held operational executive positions europolitan ericsson sas american express card travel divisions well nyman schultz ceo vodafone sweden ceo gambro renal products ceo
1805,"The group intends to relocate warehouse and office space in Loudeac and Saint Marcel to Morvillars , in the east of the country , where it already operates a hook manufacturing and distribution unit .",1,group intends to relocate warehouse office space loudeac saint marcel to morvillars east country already operates hook manufacturing distribution unit
1806,"The contract includes software licences , application maintenance and training .",1,contract includes software licences application maintenance training


In [10]:
df1 = df[['cleaned','labels']]
df1.rename(columns = {'cleaned':'sentences'}, inplace = True)


In [11]:
df1

,sentences,labels
0,merrill lynch analyst campbell morgan upgraded recommendation paperlinx from neutral to buy may,2
1,eriikka s+âderstr+âm previously held several positions finance control nokia networks including acting business group controller corporate controller position nokia siemens networks,1
2,webcast may followed online company website www.ruukki.com/investors,1
3,typical end-uses include roof structures floorings walls ceilings non-visible structures vehicles packaging boxes construction site structures fencing shelters formwork limited number concrete pourings,1
4,sale finalized september october company said,1
...,...,...
1803,appointment kaupthing bank aims to further co-ordinate capital markets activities within group to improve overall service to clients,2
1804,jon risfelt previously held operational executive positions europolitan ericsson sas american express card travel divisions well nyman schultz ceo vodafone sweden ceo gambro renal products ceo,1
1805,group intends to relocate warehouse office space loudeac saint marcel to morvillars east country already operates hook manufacturing distribution unit,1
1806,contract includes software licences application maintenance training,1


In [12]:
import spacy

In [13]:
#tokenization
tok = spacy.load('en_core_web_sm')
def tokenize (text):
    text = str(text)
    text_ = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [15]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
# import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [17]:
#count number of occurences of each word
counts = Counter()
for index, row in df1.iterrows():
    counts.update(tokenize(row['sentences']))

In [18]:
#deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 4747
num_words after: 2224


In [19]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [20]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [21]:
df1['encoded'] = df1['sentences'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df1.head()

,sentences,labels,encoded
0,merrill lynch analyst campbell morgan upgraded recommendation paperlinx from neutral to buy may,2,"[[2, 3, 4, 1, 5, 6, 7, 1, 8, 9, 10, 9, 11, 9, 12, 9, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 17]"
1,eriikka s+âderstr+âm previously held several positions finance control nokia networks including acting business group controller corporate controller position nokia siemens networks,1,"[[1, 14, 1, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 1, 25, 26, 27, 28, 27, 29, 22, 30, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 23]"
2,webcast may followed online company website www.ruukki.com/investors,1,"[[31, 13, 32, 33, 34, 35, 36, 37, 38, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 10]"
3,typical end-uses include roof structures floorings walls ceilings non-visible structures vehicles packaging boxes construction site structures fencing shelters formwork limited number concrete pourings,1,"[[1, 40, 41, 42, 43, 44, 1, 1, 1, 45, 1, 44, 46, 47, 48, 49, 50, 44, 1, 1, 1, 51, 52, 53, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 25]"
4,sale finalized september october company said,1,"[[54, 55, 56, 57, 34, 58, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 6]"


In [22]:
Counter(df1['labels'])

Counter({2: 456, 1: 1110, 0: 242})

In [23]:
X = list(df1['encoded'])
y = list(df1['labels'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [24]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [25]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [26]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [27]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [28]:
for x, y, l in train_dl:
    print('x',x)
    print('y',y)
    print('l',l)
    break

x tensor([[ 299,  189,  163,  ...,    0,    0,    0],
        [1068,  194,  398,  ...,    0,    0,    0],
        [  59,  185,   26,  ...,    0,    0,    0],
        ...,
        [  59,  643,  695,  ...,    0,    0,    0],
        [1764,  275,    1,  ...,    0,    0,    0],
        [1809, 1863, 1860,  ...,    0,    0,    0]], dtype=torch.int32)
y tensor([1, 1, 2,  ..., 0, 1, 2])
l tensor([12, 13, 17,  ..., 24, 15, 11])


In [29]:
class Stacked_RNN(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.RNN(embedding_dim, hidden_dim,n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 3)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        #out_pack, (ht, ct) = self.lstm(x_pack)
        out_pack, ht = self.rnn(x_pack)
        out = self.linear(ht[-1])
        return out

In [30]:
model = Stacked_RNN(vocab_size, 200, 100,4)

In [31]:
train_model(model, epochs=30, lr=0.01)

train loss 0.905, val loss 1.346, val accuracy 0.318, and val rmse 1.023
train loss 0.780, val loss 0.776, val accuracy 0.696, and val rmse 0.776
train loss 0.519, val loss 0.789, val accuracy 0.702, and val rmse 0.778
train loss 0.386, val loss 0.974, val accuracy 0.691, and val rmse 0.758
train loss 0.283, val loss 1.101, val accuracy 0.671, and val rmse 0.823
train loss 0.220, val loss 1.212, val accuracy 0.666, and val rmse 0.846


In [35]:
class Stacked_LSTM(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 3)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        # out_pack, ht = self.rnn(x_pack)
        out = self.linear(ht[-1])
        return out

In [36]:
model = Stacked_LSTM(vocab_size, 200, 100,4)

In [37]:
train_model(model, epochs=30, lr=0.01)

train loss 0.964, val loss 1.382, val accuracy 0.622, and val rmse 0.615
train loss 0.739, val loss 0.723, val accuracy 0.754, and val rmse 0.685
train loss 0.592, val loss 0.745, val accuracy 0.729, and val rmse 0.715
train loss 0.483, val loss 0.680, val accuracy 0.715, and val rmse 0.780
train loss 0.390, val loss 0.743, val accuracy 0.735, and val rmse 0.723
train loss 0.355, val loss 0.691, val accuracy 0.754, and val rmse 0.721
